In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Uterine_Corpus_Endometrial_Carcinoma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Review subdirectories to find one related to Uterine Corpus Endometrial Carcinoma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)
print(f"Available TCGA directories: {tcga_dirs}")

# Look for directories related to Uterine Corpus Endometrial Carcinoma
relevant_dirs = []
for dir_name in tcga_dirs:
    dir_lower = dir_name.lower()
    # Check for endometrial/endometrioid cancer first (most specific match)
    if "endometri" in dir_lower:
        relevant_dirs.append(dir_name)
    # Then check for general uterine corpus cancers
    elif "uterine" in dir_lower and "corpus" in dir_lower:
        if dir_name not in relevant_dirs:
            relevant_dirs.append(dir_name)
    # Fall back to other uterine cancers if needed
    elif "uterine" in dir_lower and len(relevant_dirs) == 0:
        relevant_dirs.append(dir_name)

print(f"Potential relevant directories for {trait}: {relevant_dirs}")

# Select the appropriate directory for the trait - prioritize endometrial/endometrioid matches
selected_dir = None
if relevant_dirs:
    # Find the best match - endometrioid/endometrial cancer is preferred over carcinosarcoma
    for dir_name in relevant_dirs:
        if "endometri" in dir_name.lower():
            selected_dir = dir_name
            break
    
    # If no endometrial-specific directory found, use the first match
    if not selected_dir:
        selected_dir = relevant_dirs[0]

if selected_dir:
    print(f"Selected directory for {trait}: {selected_dir}")
    
    # Get the full path to the directory
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Find clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {clinical_file_path}")
    print(f"Genetic data file: {genetic_file_path}")
    
    # Step 3: Load the data files
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Step 4: Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check if both datasets are available
    is_gene_available = not genetic_df.empty
    is_trait_available = not clinical_df.empty
    
    # Initial validation
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
else:
    print(f"No directory specifically relevant to the trait: {trait}")
    
    # Since the trait is not directly represented, we should record this fact
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")


Available TCGA directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_


Clinical data columns:
['CDE_ID_3226963', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_DNAMethyl_UCEC', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_UCEC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_completion_success_outcome', 'age_at_initial_pathologic_diagnosis', 'aln_pos_ihc', 'aln_pos_light_micro', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'birth_control_pill_history_usage_category', 'clinical_stage', 'colorectal_cancer', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_last_known_alive', 'days_to_new_tumor_event_additional_surge

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify potential age-related columns
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]

# Identify potential gender-related columns
candidate_gender_cols = ["gender"]

# Get the path to the clinical data file
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Endometrioid_Cancer_(UCEC)")
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data file
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview candidate age columns
if candidate_age_cols:
    age_preview = clinical_df[candidate_age_cols].head(5).to_dict()
    print("Age columns preview:")
    print(age_preview)

# Extract and preview candidate gender columns
if candidate_gender_cols:
    gender_preview = clinical_df[candidate_gender_cols].head(5).to_dict()
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': {'TCGA-2E-A9G8-01': 59.0, 'TCGA-4E-A92E-01': 54.0, 'TCGA-5B-A90C-01': 69.0, 'TCGA-5S-A9Q8-01': 51.0, 'TCGA-A5-A0G1-01': 67.0}, 'days_to_birth': {'TCGA-2E-A9G8-01': nan, 'TCGA-4E-A92E-01': -19818.0, 'TCGA-5B-A90C-01': -25518.0, 'TCGA-5S-A9Q8-01': -18785.0, 'TCGA-A5-A0G1-01': -24477.0}}
Gender columns preview:
{'gender': {'TCGA-2E-A9G8-01': 'FEMALE', 'TCGA-4E-A92E-01': 'FEMALE', 'TCGA-5B-A90C-01': 'FEMALE', 'TCGA-5S-A9Q8-01': 'FEMALE', 'TCGA-A5-A0G1-01': 'FEMALE'}}


### Step 3: Select Demographic Features

In [4]:
# Selecting columns for age and gender information

# AGE SELECTION
# Examining the age column candidates:
# 1. 'age_at_initial_pathologic_diagnosis': Has numerical values directly representing age
# 2. 'days_to_birth': Appears to have negative values representing days before birth (some missing values)
# 
# The 'age_at_initial_pathologic_diagnosis' column has direct age values and appears complete,
# so it's the better choice for age information.

age_col = 'age_at_initial_pathologic_diagnosis'

# GENDER SELECTION
# There's only one gender column candidate: 'gender'
# The preview shows it has values ('FEMALE' entries) without missing data
gender_col = 'gender'

# Print chosen columns and their information
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Endometrioid Cancer directory identified in Step 1
selected_dir = "TCGA_Endometrioid_Cancer_(UCEC)"  # Use the correct directory from Step 1
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Convert gender values from text (FEMALE/MALE) to numeric (0/1)
if 'Gender' in clinical_features.columns:
    clinical_features['Gender'] = clinical_features['Gender'].apply(tcga_convert_gender)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
# Note that all samples being positive for the trait is expected in TCGA data
note = "The dataset contains gene expression data along with clinical information for uterine endometrioid cancer patients from TCGA. All samples are positive for the trait which is expected in this dataset."

# Even if trait_biased is True, this is expected in TCGA data where all samples have the disease
# We'll consider the data usable as long as it has sufficient samples and gene expression data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=len(normalized_gene_df) > 0,
    is_trait_available=True,
    is_biased=False,  # Override trait_biased as this is expected for TCGA data
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 596 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (201, 19851) (samples x features)


After handling missing values, data shape: (201, 19850)
For the feature 'Uterine_Corpus_Endometrial_Carcinoma', the least common label is '0' with 24 occurrences. This represents 11.94% of the dataset.
The distribution of the feature 'Uterine_Corpus_Endometrial_Carcinoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 65.24598930481284
  75%: 72.0
Min: 33.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Uterine_Corpus_Endometrial_Carcinoma/TCGA.csv
